<a href="https://colab.research.google.com/github/Akshata4/llms/blob/main/multiagents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Simple Multi-Agent Homework Chatbot

class MathAgent:
    def answer(self, question):
        # very simple logic, can be expanded
        if "add" in question or "+" in question:
            try:
                # evaluate small math expressions safely
                result = eval(question.split("add")[-1])
                return f"The answer is {result}"
            except:
                return "I can help with math problems like addition or multiplication!"
        return "I'm your Math helper! Ask me about equations or numbers."

class ScienceAgent:
    def answer(self, question):
        if "planet" in question:
            return "There are 8 planets in our solar system!"
        elif "water" in question:
            return "Water is made of hydrogen and oxygen (H₂O)."
        return "I'm your Science helper! Ask me about biology, physics, or chemistry."

class EnglishAgent:
    def answer(self, question):
        if "noun" in question:
            return "A noun is a word that names a person, place, or thing."
        elif "verb" in question:
            return "A verb shows an action or a state of being."
        return "I'm your English helper! Ask me about grammar or writing."

# Main Chatbot orchestrator
class HomeworkChatbot:
    def __init__(self):
        self.agents = {
            "math": MathAgent(),
            "science": ScienceAgent(),
            "english": EnglishAgent()
        }

    def route_question(self, question):
        question = question.lower()
        # Simple keyword-based routing
        if any(word in question for word in ["math", "add", "number", "+", "multiply"]):
            return self.agents["math"].answer(question)
        elif any(word in question for word in ["science", "planet", "water", "chemistry", "physics"]):
            return self.agents["science"].answer(question)
        elif any(word in question for word in ["english", "noun", "verb", "grammar", "essay"]):
            return self.agents["english"].answer(question)
        else:
            return "I'm not sure which subject that is — can you specify Math, Science, or English?"

# Run chatbot
if __name__ == "__main__":
    bot = HomeworkChatbot()
    print("🤖 Homework Helper Bot: Ask me your question! (type 'quit' to exit)\n")

    while True:
        user_input = input("You: ")
        if user_input.lower() == "quit":
            break
        print("Bot:", bot.route_question(user_input))


🤖 Homework Helper Bot: Ask me your question! (type 'quit' to exit)

You: add 5 and 4
Bot: The answer is 4
You: science
Bot: I'm your Science helper! Ask me about biology, physics, or chemistry.
You: noun
Bot: A noun is a word that names a person, place, or thing.
You: add
Bot: I can help with math problems like addition or multiplication!
You: 4+5
Bot: The answer is 9
You: quit


In [ ]:
!pip install langchain langchain-community langchain-openai


INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32

In [8]:
import os
os.environ["OPENAI_API_KEY"] = "api_key_here"


In [ ]:
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
import os

# Set the environment variable within the script
# Replace 'YOUR_API_KEY' with your actual API key or a variable holding it
# Example: os.environ["OPENAI_API_KEY"] = 'YOUR_API_KEY'

# If you are using Colab Secrets Manager, you can access your API key like this:
# from google.colab import userdata
# os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')


# === Subject-Specific Agents ===
class BaseAgent:
    def __init__(self, subject):
        self.subject = subject
        self.llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)

    def answer(self, question):
        """Agent specialized in a given subject."""
        system_prompt = f"You are a helpful {self.subject} tutor. Explain clearly and simply for students."
        messages = [
            SystemMessage(content=system_prompt),
            HumanMessage(content=question)
        ]
        return self.llm(messages).content


class MathAgent(BaseAgent):
    def __init__(self):
        super().__init__("math")


class ScienceAgent(BaseAgent):
    def __init__(self):
        super().__init__("science")


class EnglishAgent(BaseAgent):
    def __init__(self):
        super().__init__("english")


# === Router Agent (LLM decides which agent to use) ===
class RouterAgent:
    def __init__(self):
        self.llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    def choose_subject(self, question):
        """Ask LLM to route to the correct subject."""
        system_prompt = (
            "You are a routing assistant. "
            "Given a student's question, respond with ONLY ONE WORD: "
            "'math', 'science', or 'english' based on which expert should answer."
        )
        messages = [
            SystemMessage(content=system_prompt),
            HumanMessage(content=question)
        ]
        response = self.llm(messages).content.strip().lower()
        if response not in ["math", "science", "english"]:
            return "unknown"
        return response


# === Main Orchestrator ===
class HomeworkChatbot:
    def __init__(self):
        self.router = RouterAgent()
        self.agents = {
            "math": MathAgent(),
            "science": ScienceAgent(),
            "english": EnglishAgent()
        }

    def get_answer(self, question):
        subject = self.router.choose_subject(question)
        if subject == "unknown":
            return "I'm not sure which subject that question belongs to. Try mentioning Math, Science, or English."
        return self.agents[subject].answer(question)


# === Run the Chatbot ===
if __name__ == "__main__":
    bot = HomeworkChatbot()
    print("🤖 Smart Homework Helper (multi-agent): Type 'quit' to exit\n")

    while True:
        user_input = input("You: ")
        if user_input.lower() == "quit":
            break
        print("Bot:", bot.get_answer(user_input))

🤖 Smart Homework Helper (multi-agent): Type 'quit' to exit

You: history of india
Bot: ('The history of India is rich and diverse, spanning thousands of years. Here’s a simple overview of some key periods and events:\n\n### 1. **Ancient India (c. 2500 BCE - c. 500 CE)**\n   - **Indus Valley Civilization (c. 2500 - 1500 BCE)**: One of the world’s earliest urban cultures, known for its advanced cities like Harappa and Mohenjo-Daro.\n   - **Vedic Period (c. 1500 - 500 BCE)**: Marked by the arrival of the Indo-Aryans, who composed the Vedas, ancient texts that are foundational to Hinduism.\n   - **Maurya Empire (c. 322 - 185 BCE)**: Founded by Chandragupta Maurya, it became one of the largest empires in ancient India, with notable leaders like Ashoka, who promoted Buddhism.\n\n### 2. **Classical India (c. 500 - 1200 CE)**\n   - **Gupta Empire (c. 320 - 550 CE)**: Known as the "Golden Age" of India, it saw advancements in science, mathematics, art, and literature.\n   - **Buddhism and Jaini